In [1]:
%cd ..
from inference import StyleTTS2

import librosa
import IPython.display as ipd
import torch.cuda

#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'

c:\Users\catto\Desktop\Code\Project\TTS\StyleTTS2


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\catto\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


### Load models

In [2]:
config_path = "Models/Finetune_Extend/config.yml"
models_path = "Models/Finetune_Extend/current_model_120k.pth"

### Synthesize speech

Little Note:

- You don't need to add language tokens everywhere, espeak can detect and handle them automatically most of the time.

- Reference audio has a huge impact on the result. It is best to select audio around 10s long and consistent in both tone and speed.

In [3]:
speakers = {
    "id_1": {
        "path": "Demo/reference_audio/vn_1.wav",#Ref audio path
        "lang": "vi",                           #Default language
        "speed": 1.0,                           #Speaking speed
    },
    "id_2": {
        "path": "Demo/reference_audio/vn_4.wav",
        "lang": "vi",
        "speed": 1.0,
    },
}
for id in speakers:
    max_samples = 24000*30 #max 30 seconds ref audio
    print(speakers[id]['path'])
    wave, sr = librosa.load(speakers[id]['path'], sr=24000)
    audio, index = librosa.effects.trim(wave, top_db=30)
    if sr != 24000: audio = librosa.resample(audio, sr, 24000)
    if len(audio) > max_samples: audio = audio[:max_samples]
    display(ipd.Audio(audio, rate=24000, normalize=True))

Demo/reference_audio/vn_1.wav


Demo/reference_audio/vn_4.wav


In [4]:
# text = '''Đã sáu năm trôi qua kể từ khi tôi quyết định theo đuổi công việc hiện tại. Ngay cả bây giờ, tôi vẫn nhớ rõ khoảnh khắc tôi đã dồn hết tâm huyết cho cuộc hành trình vĩ đại này. Tôi bắt đầu bằng cách rèn luyện cơ thể mình chịu đựng gian khổ. Tôi đã cùng những người săn cá voi tham gia vào một số chuyến thám hiểm đến Biển Bắc; tôi tự nguyện chịu đựng cái lạnh, cơn đói, cơn khát và thiếu ngủ; tôi làm việc chăm chỉ hơn những thủy thủ khác vào ban ngày và dành cả đêm để học toán học, y học, cũng như những các nhánh khoa học tự nhiên khác mà một nhà thám hiểm hàng hải có thể vận dụng để đạt thành tựu lớn nhất. Hai lần, tôi đã tự nguyện gia nhập làm phó dưới trên một con tàu săn cá voi tại Greenland, và tôi đã thể hiện bản thân một cách khiến người khác ngưỡng mộ. Tôi phải thừa nhận rằng, tôi cảm thấy hơi tự hào khi thuyền trưởng đã đề nghị cho tôi vị trí cao thứ hai trên tàu và khẩn cầu tôi ở lại với sự nhiệt thành, bởi ông rất coi trọng năng lực của tôi.'''

# text = '''
# [id_1][en-us]{What's up hommie}, dạo này đang học tí [en-us]{English}. Thấy bảo [en-us]{Building a strong vocabulary} khá là quan trọng. [en-us]{Bro} thấy sao?

# [id_2][en-us]{That's right}, tôi thấy [en-us]{bro} nên bắt đầu với việc đọc sách và báo tiếng Anh để quen với cách sử dụng từ, cũng như tập trung vào [en-us]{listening exercises} để cải thiện khả năng nghe.

# [id_1]Nghe nói rằng [en-us]{speaking practice} là bước quan trọng để giao tiếp tự tin. [en-us]{Bro} có muốn luyện tập với tôi không?

# [id_2][en-us]{For sure my hommie} à, cứ cho mình cái hẹn nhé.
# '''

text = '''
Việt Nam và Mỹ sẽ đàm phán một thỏa thuận thương mại song phương.
Vài giờ sau khi mức thuế đối ứng cao được tạm hoãn, Việt Nam - Mỹ đạt thống nhất sẽ đàm phán thỏa thuận thương mại song phương, trong đó có nội dung thuế.
Trong chuyến thăm và làm việc tại Mỹ ngày 10/4 (giờ Mỹ) tại Washington, Đặc phái viên của Tổng Bí thư, Phó thủ tướng Hồ Đức Phớc đã gặp Bộ trưởng Tài chính Mỹ Scott Bessent và Bộ trưởng Thương mại Howard Lutnick.
'''

In [5]:
model             = StyleTTS2(config_path, models_path).eval().to(device)
default_speaker   = "[id_1]"  #INT    Default speaker used when no speaker_id is provided in the input
avg_style         = True      #BOOL   Split the ref audio and calculate the avg styles.
stabilize         = True      #BOOL   Stabilize speaking speed.
denoise           = 0.6       #FLOAT  Adjust the strength of the denoiser. Value range is [0, 1]
n_merge           = 16        #INT    Avoid short sentences by merging when a sentence has fewer than n words

c:\Users\catto\anaconda3\Lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
c:\Users\catto\anaconda3\Lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


decoder : 54289492
predictor : 16194612
text_encoder : 5612032
style_encoder : 13845440

Total : 89941576


In [6]:
with torch.no_grad():
    r = model.generate(text, speakers, avg_style, stabilize, denoise, n_merge, default_speaker)

print('Synthesized:')
display(ipd.Audio(r, rate=24000, normalize=True))

Computing the style for: Demo/reference_audio/vn_1.wav
Computing the style for: Demo/reference_audio/vn_4.wav
Generating Audio...
Synthesized:
